In [1]:
import pandas as pd
import numpy as np

In [284]:
lenta_q = pd.read_excel('lenta_q.xlsx')

In [285]:
magnit_q = pd.read_excel('magnit_q.xlsx')

In [286]:
pyaterka_q = pd.read_excel('pyaterka_q.xlsx')

# **Кодирование осведомленности**

**Перевод частоты посещений и доли расходов в шкалу 1-5**

In [265]:
stm_share_map = {
    "Менее 10%": 1,
    "10-25%": 2,
    "26-50%": 3,
    "50-75%": 4,
    "Более 75%": 5,
}

def recode_stm_share(s):
    return s.map(stm_share_map).astype("Int64")

for df in [magnit_q, lenta_q, pyaterka_q]:
    df["stm_share_5"] = recode_stm_share(df["stm_share"])

In [266]:
stm_freq_map = {
    "Реже раза в месяц": 1,
    "Раз в месяц": 2,
    "Раз в неделю": 3,
    "Несколько раз в неделю": 4,
    "Каждый день": 5,
}

def recode_stm_freq(s):
    return s.map(stm_freq_map).astype("Int64")

for df in [magnit_q, lenta_q, pyaterka_q]:
    df["stm_freq_5"] = recode_stm_freq(df["stm_freq"])

In [267]:
# sanity-check
for name, df in {
    "magnit": magnit_q,
    "lenta": lenta_q,
    "pyaterka": pyaterka_q,
}.items():
    print("\n", name)
    print("stm_share_5:", sorted(df["stm_share_5"].dropna().unique()))
    print("stm_freq_5 :", sorted(df["stm_freq_5"].dropna().unique()))


 magnit
stm_share_5: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
stm_freq_5 : [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]

 lenta
stm_share_5: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
stm_freq_5 : [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]

 pyaterka
stm_share_5: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
stm_freq_5 : [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]


**Доля правильно названных брендов -> 1-5**

In [268]:
def norm(x):
    if pd.isna(x):
        return np.nan
    return str(x).strip().lower()


In [269]:
import numpy as np

q_cols = [
    "logo_matritsa", "logo_moya_tsena", "logo_lenta_fresh",
    "logo_primiere_of_taste", "logo_m_kuhnya", "logo_green_ribbon",
    "logo_lucky_days", "logo_m", "logo_gusto_di_roma", "logo_m_magnit"
]

# 1) нормализация
for c in q_cols:
    magnit_q[c] = magnit_q[c].map(norm)

KNOWN_ALLOWED = {norm("Бренды известных производителей")}

MAGNIT_PL_ALLOWED = {
    norm("СТМ Магнит семейный"),
    norm("Фантазийная СТМ Магнит семейный"),
    norm("Фантазийные СТМ Магнит семейный"),
}

OTHER_NETWORK_ALLOWED = {norm("СТМ другой сети")}

question_type = {
    "logo_matritsa": "KNOWN",
    "logo_moya_tsena": "MAGNIT_PL",
    "logo_lenta_fresh": "OTHER_NETWORK",
    "logo_primiere_of_taste": "MAGNIT_PL",
    "logo_m_kuhnya": "MAGNIT_PL",
    "logo_green_ribbon": "MAGNIT_PL",
    "logo_lucky_days": "MAGNIT_PL",
    "logo_m": "MAGNIT_PL",
    "logo_gusto_di_roma": "MAGNIT_PL",
    "logo_m_magnit": "MAGNIT_PL",
}

# 2) флаги — ВАЖНО: пишем в magnit_q
for q in q_cols:
    flag = f"{q}_flag"
    qtype = question_type[q]

    if qtype == "KNOWN":
        allowed = KNOWN_ALLOWED
    elif qtype == "MAGNIT_PL":
        allowed = MAGNIT_PL_ALLOWED
    elif qtype == "OTHER_NETWORK":
        allowed = OTHER_NETWORK_ALLOWED
    else:
        raise ValueError(f"Unknown question_type for {q}: {qtype}")

    magnit_q[flag] = np.where(
        magnit_q[q].isna(),
        np.nan,
        magnit_q[q].isin(allowed).astype(int)
    )

# 3) чек
flag_cols = [f"{q}_flag" for q in q_cols]
print("Created flags:", [c for c in flag_cols if c in magnit_q.columns])
print(magnit_q[flag_cols].mean().sort_index())



Created flags: ['logo_matritsa_flag', 'logo_moya_tsena_flag', 'logo_lenta_fresh_flag', 'logo_primiere_of_taste_flag', 'logo_m_kuhnya_flag', 'logo_green_ribbon_flag', 'logo_lucky_days_flag', 'logo_m_flag', 'logo_gusto_di_roma_flag', 'logo_m_magnit_flag']
logo_green_ribbon_flag         0.147287
logo_gusto_di_roma_flag        0.116279
logo_lenta_fresh_flag          0.767442
logo_lucky_days_flag           0.298450
logo_m_flag                    0.883721
logo_m_kuhnya_flag             0.856589
logo_m_magnit_flag             0.906977
logo_matritsa_flag             0.655039
logo_moya_tsena_flag           0.655039
logo_primiere_of_taste_flag    0.143411
dtype: float64


In [270]:
q_cols = [
    "logo_dobryy", "logo_365_dney", "logo_giardino_club", "logo_spar",
    "logo_lenta", "logo_delisse", "logo_premium_club", "logo_dolce_albero",
    "logo_home_club", "logo_lenta_fresh"
]

# 1) нормализуем ответы
for c in q_cols:
    lenta_q[c] = lenta_q[c].map(norm)

# 2) allowed-наборы (на всякий: добавил и "фантазийные", и "фантазийная" — потому что анкеты любят плясать формулировками)
KNOWN_ALLOWED = {norm("Бренды известных производителей")}

LENTA_PL_ALLOWED = {
    norm("СТМ Лента"),
    norm("Фантазийная СТМ Лента"),
    norm("Фантазийные СТМ Лента"),
}

OTHER_NETWORK_ALLOWED = {norm("СТМ другой сети")}

# 3) маппинг вопрос -> тип правильного ответа (по файлу/скрину)
question_type = {
    "logo_dobryy": "KNOWN",
    "logo_365_dney": "LENTA_PL",
    "logo_giardino_club": "LENTA_PL",
    "logo_spar": "OTHER_NETWORK",
    "logo_lenta": "LENTA_PL",
    "logo_delisse": "LENTA_PL",
    "logo_premium_club": "LENTA_PL",
    "logo_dolce_albero": "LENTA_PL",
    "logo_home_club": "LENTA_PL",
    "logo_lenta_fresh": "LENTA_PL",
}

# 4) считаем флаги
for q in q_cols:
    flag = f"{q}_flag"

    if question_type[q] == "KNOWN":
        allowed = KNOWN_ALLOWED
    elif question_type[q] == "LENTA_PL":
        allowed = LENTA_PL_ALLOWED
    elif question_type[q] == "OTHER_NETWORK":
        allowed = OTHER_NETWORK_ALLOWED
    else:
        raise ValueError(f"Unknown question_type for {q}: {question_type[q]}")

    lenta_q[flag] = np.where(
        lenta_q[q].isna(),
        np.nan,
        lenta_q[q].isin(allowed).astype(int)
    )

# 5) быстрый чек
flag_cols = [f"{q}_flag" for q in q_cols]
print(lenta_q[flag_cols].mean().sort_index())


logo_365_dney_flag         0.507752
logo_delisse_flag          0.162791
logo_dobryy_flag           0.682171
logo_dolce_albero_flag     0.151163
logo_giardino_club_flag    0.166667
logo_home_club_flag        0.279070
logo_lenta_flag            0.941860
logo_lenta_fresh_flag      0.891473
logo_premium_club_flag     0.158915
logo_spar_flag             0.457364
dtype: float64


In [271]:


q_cols = [
    "logo_dobryy",
    "logo_global_village",
    "logo_krasnaya_tsena",
    "logo_restoriya",
    "logo_selyanochka",
    "logo_fish_house",
    "logo_mix_bar",
    "logo_stantsiya_molochnaya",
    "logo_sarafanovo",
    "logo_lenta_fresh",
]

# 1) нормализуем ответы
for c in q_cols:
    pyaterka_q[c] = pyaterka_q[c].map(norm)

# 2) allowed-наборы (ТОЧНЫЕ формулировки анкеты)
KNOWN_ALLOWED = {norm("Бренды известных производителей")}

PYATERKA_PL_ALLOWED = {
    norm("СТМ Пятерочка"),
    norm("Фантазийная СТМ Пятерочка"),
}

OTHER_NETWORK_ALLOWED = {
    norm("СТМ другой сети"),
}

# 3) маппинг вопрос -> тип правильного ответа (по файлу)
question_type = {
    "logo_dobryy": "KNOWN",
    "logo_global_village": "PYATERKA_PL",
    "logo_krasnaya_tsena": "PYATERKA_PL",
    "logo_restoriya": "PYATERKA_PL",
    "logo_selyanochka": "PYATERKA_PL",
    "logo_fish_house": "PYATERKA_PL",
    "logo_mix_bar": "PYATERKA_PL",
    "logo_stantsiya_molochnaya": "PYATERKA_PL",
    "logo_sarafanovo": "PYATERKA_PL",
    "logo_lenta_fresh": "OTHER_NETWORK",
}

# 4) считаем флаги
for q in q_cols:
    flag = f"{q}_flag"

    if question_type[q] == "KNOWN":
        allowed = KNOWN_ALLOWED
    elif question_type[q] == "PYATERKA_PL":
        allowed = PYATERKA_PL_ALLOWED
    elif question_type[q] == "OTHER_NETWORK":
        allowed = OTHER_NETWORK_ALLOWED
    else:
        raise ValueError(f"Unknown question_type for {q}: {question_type[q]}")

    pyaterka_q[flag] = np.where(
        pyaterka_q[q].isna(),
        np.nan,
        pyaterka_q[q].isin(allowed).astype(int)
    )

# 5) sanity-check
flag_cols = [f"{q}_flag" for q in q_cols]
print(pyaterka_q[flag_cols].mean().sort_index())


logo_dobryy_flag                  0.693798
logo_fish_house_flag              0.147287
logo_global_village_flag          0.496124
logo_krasnaya_tsena_flag          0.709302
logo_lenta_fresh_flag             0.759690
logo_mix_bar_flag                 0.186047
logo_restoriya_flag               0.457364
logo_sarafanovo_flag              0.104651
logo_selyanochka_flag             0.255814
logo_stantsiya_molochnaya_flag    0.232558
dtype: float64


In [272]:
magnit_flag_cols = [c for c in magnit_q.columns if c.endswith("_flag")]

magnit_q["brand_knowledge_score"] = (
    magnit_q[magnit_flag_cols]
    .fillna(0)
    .sum(axis=1)
)

In [273]:
lenta_flag_cols = [c for c in lenta_q.columns if c.endswith("_flag")]

lenta_q["brand_knowledge_score"] = (
    lenta_q[lenta_flag_cols]
    .fillna(0)
    .sum(axis=1)
)


In [274]:
pyaterka_flag_cols = [c for c in pyaterka_q.columns if c.endswith("_flag")]

pyaterka_q["brand_knowledge_score"] = (
    pyaterka_q[pyaterka_flag_cols]
    .fillna(0)
    .sum(axis=1)
)


In [275]:
magnit_q["brand_knowledge_score"].describe()
lenta_q["brand_knowledge_score"].describe()
pyaterka_q["brand_knowledge_score"].describe()

count    258.000000
mean       4.042636
std        2.126844
min        0.000000
25%        3.000000
50%        4.000000
75%        5.000000
max       10.000000
Name: brand_knowledge_score, dtype: float64

In [276]:
lenta_q["brand_knowledge_score"].describe()

count    258.000000
mean       4.399225
std        1.837652
min        0.000000
25%        3.000000
50%        4.000000
75%        5.000000
max       10.000000
Name: brand_knowledge_score, dtype: float64

In [277]:
magnit_q["brand_knowledge_score"].describe()

count    258.000000
mean       5.430233
std        1.824225
min        0.000000
25%        5.000000
50%        5.000000
75%        6.000000
max       10.000000
Name: brand_knowledge_score, dtype: float64

**Подготовка шкал Лайкерта**


In [278]:
def to_5point_scale(s):
    return pd.cut(
        s,
        bins=[-1, 2, 4, 6, 8, 10],
        labels=[1, 2, 3, 4, 5]
    ).astype("int")

In [279]:
magnit_q["stm_knowledge"] = to_5point_scale(magnit_q["brand_knowledge_score"])
lenta_q["stm_knowledge"] = to_5point_scale(lenta_q["brand_knowledge_score"])
pyaterka_q["stm_knowledge"] = to_5point_scale(pyaterka_q["brand_knowledge_score"])

In [280]:
lenta_q = lenta_q.rename(columns={
    "Я регулярно совершаю покупки в магазинах Лента":
        "loyalty_retailer_regular_purchase_likert",
    "Когда есть выбор я предпочитаю магазины Лента другим магазинам":
        "loyalty_retailer_prefer_over_brands_likert",
    "recommend_chain":
        "loyalty_retailer_recommend_pl_likert",
})
magnit_q = magnit_q.rename(columns={
    'Я регулярно совершаю покупки в магазинах\xa0супермаркетов Магнит Семейный':
        "loyalty_retailer_regular_purchase_likert",
    "Когда есть выбор я предпочитаю магазины супермаркетов Магнит Семейный другим магазинам":
        "loyalty_retailer_prefer_over_brands_likert",
    "recommend_chain":
        "loyalty_retailer_recommend_pl_likert",
})
pyaterka_q = pyaterka_q.rename(columns={
    "loyalty_retailer_item_regular_purchase_likert":
        "loyalty_retailer_regular_purchase_likert",
    "prefer_chain":
        "loyalty_retailer_prefer_over_brands_likert",
    "recommend_chain":
        "loyalty_retailer_recommend_pl_likert",
})


In [288]:
import pandas as pd
import numpy as np

ATTITUDE_COLS = [
    "attitude_item_prefer_available_likert",
    "attitude_item_brand_alternative_likert",
    "attitude_item_quality_concern_likert",
]

ATTITUDE_ALLOWED = {
    "1- совершенно не согласен",
    "2- скорее не согласен",
    "4- скорее согласен",
    "5- полностью согласен",
}

def norm_text(x):
    if pd.isna(x):
        return ""
    return str(x).strip().lower()

# Нормализуем allowed один раз (на случай регистра/пробелов)
ATTITUDE_ALLOWED_NORM = {norm_text(v) for v in ATTITUDE_ALLOWED}

def encode_attitude_value(x):
    s = norm_text(x)
    # если это один из "нормальных" вариантов 1/2/4/5 — берем первую цифру
    if s in ATTITUDE_ALLOWED_NORM and s and s[0].isdigit():
        return int(s[0])
    # все остальное (включая 'З-...') — это 3
    return 3

def encode_attitude_as_midpoint(df):
    for c in ATTITUDE_COLS:
        if c in df.columns:
            df[c] = df[c].apply(encode_attitude_value).astype("Int64")
            # на всякий (мусор)

In [289]:
def likert_first_digit_or_na(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()
    if s and s[0].isdigit():
        return int(s[0])
    return pd.NA

def encode_likert_except_attitude(df):
    likert_cols = [c for c in df.columns if c.endswith("_likert") and c not in ATTITUDE_COLS]
    for c in likert_cols:
        df[c] = df[c].apply(likert_first_digit_or_na).astype("Int64")

def run_encoding(df, name):
    encode_attitude_as_midpoint(df)
    encode_likert_except_attitude(df)

    print(f"\n{name}: attitude uniques after encoding")
    for c in ATTITUDE_COLS:
        if c in df.columns:
            print(c, sorted(df[c].dropna().unique()))

    # чек: в attitude NA быть не должно
    cols = [c for c in ATTITUDE_COLS if c in df.columns]
    if cols:
        na_share = df[cols].isna().mean()
        if (na_share > 0).any():
            raise ValueError(f"{name}: attitude still has NA:\n{na_share}")

run_encoding(magnit_q, "MAGNIT")
run_encoding(lenta_q, "LENTA")
run_encoding(pyaterka_q, "PYATERKA")


MAGNIT: attitude uniques after encoding
attitude_item_prefer_available_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
attitude_item_brand_alternative_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
attitude_item_quality_concern_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]

LENTA: attitude uniques after encoding
attitude_item_prefer_available_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
attitude_item_brand_alternative_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
attitude_item_quality_concern_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]

PYATERKA: attitude uniques after encoding
attitude_item_prefer_available_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
attitude_item_brand_alternative_likert [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
attitude_item_quality_concern_likert [np.in

In [96]:
magnit_q.to_excel("magnit_cfa.xlsx", index=False)
lenta_q.to_excel("lenta_cfa.xlsx", index=False)
pyaterka_q.to_excel("pyaterka_cfa.xlsx", index=False)

In [97]:
# 1) добавим идентификатор сети (супер полезно для CFA/инвариантности)
magnit_q2 = magnit_q.copy()
lenta_q2 = lenta_q.copy()
pyaterka_q2 = pyaterka_q.copy()

magnit_q2["retailer"] = "magnit"
lenta_q2["retailer"] = "lenta"
pyaterka_q2["retailer"] = "pyaterka"

# 2) выравниваем набор колонок (на случай, если где-то колонок больше/меньше)
all_cols = sorted(set(magnit_q2.columns) | set(lenta_q2.columns) | set(pyaterka_q2.columns))

magnit_q2 = magnit_q2.reindex(columns=all_cols)
lenta_q2 = lenta_q2.reindex(columns=all_cols)
pyaterka_q2 = pyaterka_q2.reindex(columns=all_cols)

# 3) вертикальная склейка
all_q = pd.concat([magnit_q2, lenta_q2, pyaterka_q2], axis=0, ignore_index=True)

# 4) выгрузка
out_path = "all_retailers_stacked.xlsx"
all_q.to_excel(out_path, index=False)

print("Saved:", out_path, "shape:", all_q.shape)

Saved: all_retailers_stacked.xlsx shape: (774, 94)


In [98]:
magnit_q.columns

Index(['visit_freq', 'logo_matritsa', 'logo_moya_tsena', 'logo_lenta_fresh',
       'logo_primiere_of_taste', 'logo_m_kuhnya', 'logo_green_ribbon',
       'logo_lucky_days', 'logo_m', 'logo_gusto_di_roma', 'logo_m_magnit',
       'stm_freq', 'stm_share', 'attitude_item_quality_concern_likert',
       'attitude_item_prefer_available_likert',
       'attitude_item_brand_alternative_likert',
       'benefit_item_saves_money_likert',
       'benefit_item_choose_same_price_likert',
       'benefit_item_value_money_likert',
       'assortment_item_category_coverage_likert',
       'assortment_item_price_range_likert',
       'assortment_item_wide_choice_likert',
       'retailer_brand_item_logo_quality_trust_likert',
       'retailer_brand_item_quality_guarantee_likert',
       'retailer_brand_item_failure_hurts_attitude_likert',
       'cannibalization_item_too_similar_likert',
       'cannibalization_item_choice_difficulty_likert',
       'cannibalization_item_segment_clarity_likert',
    

In [99]:
cols = [
    "attitude_item_brand_alternative_likert",
    "attitude_item_prefer_available_likert"
]

all_q[cols].dtypes


attitude_item_brand_alternative_likert    Int64
attitude_item_prefer_available_likert     Int64
dtype: object

In [100]:
for c in cols:
    print(f"\n=== {c} ===")
    print(all_q[c].value_counts(dropna=False).head(20))



=== attitude_item_brand_alternative_likert ===
attitude_item_brand_alternative_likert
4       328
<NA>    208
5       114
2        90
1        34
Name: count, dtype: Int64

=== attitude_item_prefer_available_likert ===
attitude_item_prefer_available_likert
4       267
<NA>    193
2       188
5        83
1        43
Name: count, dtype: Int64


In [101]:
all_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774 entries, 0 to 773
Data columns (total 94 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   assortment_item_category_coverage_likert           774 non-null    Int64  
 1   assortment_item_price_range_likert                 774 non-null    Int64  
 2   assortment_item_wide_choice_likert                 774 non-null    Int64  
 3   attitude_item_brand_alternative_likert             566 non-null    Int64  
 4   attitude_item_prefer_available_likert              581 non-null    Int64  
 5   attitude_item_quality_concern_likert               586 non-null    Int64  
 6   benefit_item_choose_same_price_likert              774 non-null    Int64  
 7   benefit_item_saves_money_likert                    774 non-null    Int64  
 8   benefit_item_value_money_likert                    774 non-null    Int64  
 9   brand_know